# 기온 예측

In [1]:
import os
import itertools
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.utils import tensorboard
from torch.utils.data import DataLoader, Dataset

random_seed = 20181401
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

display = display if "display" in vars(__builtins__) else print
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
writer = tensorboard.SummaryWriter()

display(device)

device(type='cuda', index=0)

In [2]:
learning_rate = 0.0001
training_epochs = 20
batch_size = 64

In [3]:
class MyDataset(Dataset):
  def __init__(self, root, train, target_delay=144, seq_len=720, stride=5):
    df = pd.read_csv(os.path.join(root, "jena-climate-2009-2016.csv"))
    df["Date Time"] = df["Date Time"].apply(lambda x: float(int(x.split()[1].replace(":", ""), 10) // 1000))
    full_x = torch.FloatTensor(df.drop(["Tpot (K)", "Tdew (degC)"], axis=1).values)
    full_y = torch.FloatTensor(df["T (degC)"].values).view(-1, 1)

    test_set_start_index = int(full_x.shape[0] * 0.9)
    self.train_mean = full_x[:test_set_start_index].mean(dim=0)
    self.train_std = full_x[:test_set_start_index].std(dim=0)

    if train:
      self.x = (full_x[:test_set_start_index] - self.train_mean) / self.train_std
      self.y = full_y[:test_set_start_index]
    else:
      self.x = (full_x[test_set_start_index:] - self.train_mean) / self.train_std
      self.y = full_y[test_set_start_index:]
    
    self.target_delay = target_delay
    self.seq_len = seq_len
    self.stride = stride

  def __getitem__(self, index):
    x_start = index * self.stride
    x_end = x_start + self.seq_len - 1
    return self.x[x_start:(x_end + 1)], self.y[x_end + self.target_delay]
  
  def __len__(self):
    return (self.x.shape[0] - self.seq_len - self.target_delay) // self.stride

In [4]:
train_dataset = MyDataset(root="datasets/", train=True)
test_dataset = MyDataset(root="datasets/", train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [5]:
class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.input_size = 13
    self.hidden_size = 128
    self.num_layers = 3
    self.gru = nn.GRU(self.input_size, self.hidden_size, num_layers=self.num_layers, batch_first=True)
    self.fc = nn.Sequential(
      nn.Linear(self.hidden_size, self.hidden_size // 4),
      nn.BatchNorm1d(self.hidden_size // 4),
      nn.SiLU(),
      nn.Dropout(),
      nn.Linear(self.hidden_size // 4, 1)
    )
  
  def forward(self, x):
    out, h = self.gru(x, torch.zeros((self.num_layers, x.shape[0], self.hidden_size)).to(x.device))
    out = self.fc(out[:, -1, :])
    return out

In [6]:
model = MyModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3, threshold=0.0001)

display(model)
writer.add_graph(model, next(iter(test_loader))[0].to(device))
writer.flush()

MyModel(
  (gru): GRU(13, 128, num_layers=3, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [7]:
def validate():
  with torch.no_grad():
    model.eval()
    batch_count = len(test_loader)
    avg_cost = 0

    for x, y in test_loader:
      assert isinstance(x, torch.Tensor)
      assert isinstance(y, torch.Tensor)
      x = x.to(device)
      y = y.to(device)

      hypothesis = model(x)
      cost = criterion(hypothesis, y)
      avg_cost += cost.item() / batch_count
    
    model.train()
    return avg_cost

In [8]:
display("epoch: 0 / validate cost: {}".format(validate()))
best_cost = 10**9

for epoch in range(training_epochs):
  batch_count = len(train_loader)
  avg_cost = 0

  for x, y in train_loader:
    assert isinstance(x, torch.Tensor)
    assert isinstance(y, torch.Tensor)
    model.train()
    x = x.to(device)
    y = y.to(device)

    hypothesis = model(x)
    cost = criterion(hypothesis, y)
    avg_cost += cost.item() / batch_count

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
  
  validate_cost = validate()
  display("epoch: {} / train cost: {} / validate cost: {} / lr: {}".format(epoch + 1, avg_cost, validate_cost, optimizer.param_groups[0]["lr"]))
  writer.add_scalars("Cost", {"train": avg_cost, "validate": validate_cost}, epoch + 1)
  writer.flush()

  if validate_cost < best_cost:
    best_cost = validate_cost
    torch.save(model.state_dict(), "models/rnn-weather.pth")

  scheduler.step(validate_cost)

'epoch: 0 / validate cost: 203.2954479623586'

'epoch: 1 / train cost: 121.40099177602991 / validate cost: 131.27748008444905 / lr: 0.0001'

'epoch: 2 / train cost: 88.68182259899078 / validate cost: 83.82459251023829 / lr: 0.0001'

'epoch: 3 / train cost: 58.75101846113033 / validate cost: 46.805773958563805 / lr: 0.0001'

'epoch: 4 / train cost: 36.68584143590114 / validate cost: 24.571247042156756 / lr: 0.0001'

...

'epoch: 16 / train cost: 15.304808526119938 / validate cost: 7.809668253641576 / lr: 0.0001'

'epoch: 17 / train cost: 15.036093264919216 / validate cost: 8.0796277821064 / lr: 0.0001'

'epoch: 18 / train cost: 14.828768835633499 / validate cost: 7.161372140049934 / lr: 5e-05'

'epoch: 19 / train cost: 14.693706968679251 / validate cost: 6.866065793670714 / lr: 5e-05'

'epoch: 20 / train cost: 14.484462547302243 / validate cost: 7.164786709472537 / lr: 5e-05'

In [9]:
model.load_state_dict(torch.load("models/rnn-weather.pth", map_location=device))

with torch.no_grad():
  model.eval()
  
  for n, (x, y) in enumerate(itertools.islice(DataLoader(test_dataset, batch_size=1, shuffle=True), 300)):
    assert isinstance(x, torch.Tensor)
    assert isinstance(y, torch.Tensor)
    x = x.to(device)
    y = y.to(device)
    hypothesis = model(x)
    display("index: {} / real: {:.1f}℃ / prediction: {:.1f}℃".format(n + 1, y.item(), hypothesis.item()))
  
  model.train()

writer.close()

'index: 1 / real: 18.3℃ / prediction: 15.7℃'

'index: 2 / real: 22.5℃ / prediction: 21.9℃'

'index: 3 / real: 22.4℃ / prediction: 16.7℃'

'index: 4 / real: 10.1℃ / prediction: 13.0℃'

'index: 5 / real: 5.8℃ / prediction: 4.0℃'

...

'index: 296 / real: 17.7℃ / prediction: 13.1℃'

'index: 297 / real: 5.9℃ / prediction: 2.6℃'

'index: 298 / real: 10.5℃ / prediction: 7.0℃'

'index: 299 / real: 19.7℃ / prediction: 21.9℃'

'index: 300 / real: 6.4℃ / prediction: 8.2℃'